In [ ]:
# Breast Cancer EDA — Human + Agentic Workflow

**Goal:** EDA that’s reproducible and aligned with our agentic pipeline.  
We load the dataset from the web (and freeze a local copy), explore distributions/correlations,
and then show how our agents (EDA → Modeling → Explain) operate on the same data.

In [1]:
# Basic setup for EDA
%pip install -q pandas numpy matplotlib seaborn scikit-learn shap plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
warnings.filterwarnings("ignore")


Note: you may need to restart the kernel to use updated packages.
